In [77]:
import json
import numpy as np
from matplotlib import pyplot as plt

from postprocess_original_utils import (load_data, wrongness,
    get_keypoint_comparison_depths, metaperson_comparisons, worker_comparisons,
    fraction_of_correct_comparisons_per_trial)


import pickle
import os
import sys
import itertools
import copy
import difflib
import random
from collections import Counter, defaultdict
from lean_correction import correct_lean

import matplotlib.pyplot as plt

from constants import (HUMAN_ANNOTATION_PATH, COCO_ANNOTATION_PATH,
    HUMAN_RAW_RESULT_PATH, COCO_RAW_RESULT_PATH, KEYPTS_RELATIVE_DEPTH_PATH,
    KEYCMPS_RESULT_PATH, HUMAN_OUTPUT_PATH)

with open(HUMAN_ANNOTATION_PATH) as f:
        _human_dataset = json.load(f)
#         correct_lean(_human_dataset)

data = load_data(load_from_file=False, full_ordering=False)

Read in 1000 HIT total assignments
979 good, 21 flagged, 0 error, 0 rejected
105812 images in human3.6 dataset
979 annotations matched with ground truth
Output data to /Users/Robert/Documents/Caltech/CS81_Depth_Research/results/human36m_processed_data.json


In [78]:
subj_id_to_yvals = {}
axis_to_sort = 2
for d in _human_dataset['annotations']:
    subj_id_to_yvals[d['i_id']] = list(d['kpts_3d'])
#     subj_id_to_yvals[d['i_id']][3:6] = [] # Remove neck keypoint
    subj_id_to_yvals[d['i_id']] = subj_id_to_yvals[d['i_id']][axis_to_sort::3]

In [11]:
# Fancy hybrid per trial and per assignment

fractions_of_correct_comps_per_trial = []
fractions_of_correct_comps_per_assignment = []
for d in data:
    assignment_comp_res = []
    for t in d['trials']:
        subj_id = t['human_subj_id']
        yvals = subj_id_to_yvals[subj_id]
        trial_comp_res = []
        for comp, comp_res in t['depth']['keypoint_comparisons_res'].iteritems():
            kpt1, kpt2 = [int(_k) for _k in comp.split(',')]
            kpt1_depth = yvals[kpt1]
            kpt2_depth = yvals[kpt2]
            depth_diff = kpt1_depth - kpt2_depth
            trial_comp_res.append(int(np.sign(depth_diff) == np.sign(comp_res)))
        assignment_comp_res.extend(trial_comp_res)
        fractions_of_correct_comps_per_trial.append(sum(trial_comp_res) * 1.0 / len(trial_comp_res))
    fractions_of_correct_comps_per_assignment.append(sum(assignment_comp_res) * 1.0 / len(assignment_comp_res))
    
# fractions_of_correct_comps_per_trial
# fractions_of_correct_comps_per_assignment

In [12]:
fractions_of_correct_comps_per_assignment = []

for d in data:
    assignment_comp_res = []
    for t in d['trials']:
        subj_id = t['human_subj_id']
        yvals = subj_id_to_yvals[subj_id]
        for comp, comp_res in t['depth']['keypoint_comparisons_res'].iteritems():
            kpt1, kpt2 = [int(_k) for _k in comp.split(',')]
            kpt1_depth = yvals[kpt1]
            kpt2_depth = yvals[kpt2]
            depth_diff = kpt1_depth - kpt2_depth
            assignment_comp_res.append(int(np.sign(depth_diff) == np.sign(comp_res)))
    fractions_of_correct_comps_per_assignment.append(sum(assignment_comp_res) * 1.0 / len(assignment_comp_res))

# fractions_of_correct_comps_per_assignment

In [13]:
subj_id_to_trials = {}

for d in data:
    for t in d['trials']:
        subj_id = t['human_subj_id']
        if subj_id in subj_id_to_trials:
            subj_id_to_trials[subj_id].append(t['depth']['keypoint_comparisons_res'])
        else:
            subj_id_to_trials[subj_id] = [t['depth']['keypoint_comparisons_res']]

In [14]:
fractions_of_correct_comps_per_subject = []

for subj_id in subj_id_to_trials.keys():
    subj_comp_res = []
    trials = subj_id_to_trials[subj_id]
    yvals = subj_id_to_yvals[subj_id]
    for t in trials:
        for comp, comp_res in t.iteritems():
            kpt1, kpt2 = [int(_k) for _k in comp.split(',')]
            kpt1_depth = yvals[kpt1]
            kpt2_depth = yvals[kpt2]
            depth_diff = kpt1_depth - kpt2_depth
            subj_comp_res.append(int(np.sign(depth_diff) == np.sign(comp_res)))
    fractions_of_correct_comps_per_subject.append(sum(subj_comp_res) * 1.0 / len(subj_comp_res))

In [15]:
hit_id_to_trials = {}

for d in data:
    hit_id = d['hit_id']
    if hit_id in hit_id_to_trials:
        hit_id_to_trials[hit_id].append(d['trials'])
    else:
        hit_id_to_trials[hit_id] = [d['trials']]

In [16]:
fractions_of_correct_comps_per_hit = []

for hit_id in hit_id_to_trials.keys():
    hit_comp_res = []
    trials = hit_id_to_trials[hit_id]
    for trial in trials:
        for t in trial:
            subj_id = t['human_subj_id']
            yvals = subj_id_to_yvals[subj_id]
            for comp, comp_res in t['depth']['keypoint_comparisons_res'].iteritems():
                kpt1, kpt2 = [int(_k) for _k in comp.split(',')]
                kpt1_depth = yvals[kpt1]
                kpt2_depth = yvals[kpt2]
                depth_diff = kpt1_depth - kpt2_depth
                hit_comp_res.append(int(np.sign(depth_diff) == np.sign(comp_res)))
    fractions_of_correct_comps_per_hit.append(sum(hit_comp_res) * 1.0 / len(hit_comp_res))

In [17]:
worker_id_to_trials = {}

for d in data:
    worker_id = d['worker_id']
    if worker_id in worker_id_to_trials:
        worker_id_to_trials[worker_id].append(d['trials'])
    else:
        worker_id_to_trials[worker_id] = [d['trials']]

In [18]:
fractions_of_correct_comps_per_worker = []

for worker_id in worker_id_to_trials.keys():
    worker_comp_res = []
    trials = worker_id_to_trials[worker_id]
    for trial in trials:
        for t in trial:
            subj_id = t['human_subj_id']
            yvals = subj_id_to_yvals[subj_id]
            for comp, comp_res in t['depth']['keypoint_comparisons_res'].iteritems():
                kpt1, kpt2 = [int(_k) for _k in comp.split(',')]
                kpt1_depth = yvals[kpt1]
                kpt2_depth = yvals[kpt2]
                depth_diff = kpt1_depth - kpt2_depth
                worker_comp_res.append(int(np.sign(depth_diff) == np.sign(comp_res)))
    fractions_of_correct_comps_per_worker.append(sum(worker_comp_res) * 1.0 / len(worker_comp_res))

In [19]:
import random
subj_id_to_metatrials = {}

for subj_id in subj_id_to_trials.keys():
    metatrial = {}
    for t in subj_id_to_trials[subj_id]:
        for comp, comp_res in t.iteritems():
            if comp in metatrial:
                metatrial[comp] += comp_res
            else:
                metatrial[comp] = comp_res
    for comp, comp_res in metatrial.iteritems():
        metatrial[comp] = np.sign(comp_res) if comp_res != 0 else random.choice([-1, 1])
    subj_id_to_metatrials[subj_id] = metatrial

In [20]:
fractions_of_correct_comps_per_metatrial = []

for subj_id, t in subj_id_to_metatrials.iteritems():
    yvals = subj_id_to_yvals[subj_id]
    trial_comp_res = []
    for comp, comp_res in t.iteritems():
        kpt1, kpt2 = [int(_k) for _k in comp.split(',')]
        kpt1_depth = yvals[kpt1]
        kpt2_depth = yvals[kpt2]
        depth_diff = kpt1_depth - kpt2_depth
        trial_comp_res.append(int(np.sign(depth_diff) == np.sign(comp_res)))
    fractions_of_correct_comps_per_metatrial.append(sum(trial_comp_res) * 1.0 / len(trial_comp_res))

In [21]:
hit_id_to_subj_ids = {}

for d in data:
    hit_id_to_subj_ids[d['hit_id']] = d['human_subj_ids']

In [22]:
fractions_of_correct_comps_per_metahit = []

for hit_id, subj_ids in hit_id_to_subj_ids.iteritems():
    metahit_comp_res = []
    for subj_id in subj_ids:
        t = subj_id_to_metatrials[subj_id]
        yvals = subj_id_to_yvals[subj_id]
        for comp, comp_res in t.iteritems():
            kpt1, kpt2 = [int(_k) for _k in comp.split(',')]
            kpt1_depth = yvals[kpt1]
            kpt2_depth = yvals[kpt2]
            depth_diff = kpt1_depth - kpt2_depth
            metahit_comp_res.append(int(np.sign(depth_diff) == np.sign(comp_res)))
    fractions_of_correct_comps_per_metahit.append(sum(metahit_comp_res) * 1.0 / len(metahit_comp_res))

In [23]:
workers_ranked_worst_to_best = []

worker_accuracy = {}
for worker_id in worker_id_to_trials.keys():
    worker_comp_res = []
    trials = worker_id_to_trials[worker_id]
    for trial in trials:
        for t in trial:
            subj_id = t['human_subj_id']
            yvals = subj_id_to_yvals[subj_id]
            for comp, comp_res in t['depth']['keypoint_comparisons_res'].iteritems():
                kpt1, kpt2 = [int(_k) for _k in comp.split(',')]
                kpt1_depth = yvals[kpt1]
                kpt2_depth = yvals[kpt2]
                depth_diff = kpt1_depth - kpt2_depth
                worker_comp_res.append(int(np.sign(depth_diff) == np.sign(comp_res)))
    worker_accuracy[worker_id] = sum(worker_comp_res) * 1.0 / len(worker_comp_res)
workers_ranked_worst_to_best = [key for key, value in sorted(worker_accuracy.iteritems(), key=lambda (k,v): (v,k))]

In [24]:
distance_of_wrong_comps = []
distance_of_correct_comps = []

FRACTION_OF_WORKERS_TO_EXCLUDE = 0.1
accepted_worker_ids = workers_ranked_worst_to_best[int(len(workers_ranked_worst_to_best) * FRACTION_OF_WORKERS_TO_EXCLUDE):]

for d in data:
    if d['worker_id'] not in accepted_worker_ids:
        continue
    for t in d['trials']:
        subj_id = t['human_subj_id']
        yvals = subj_id_to_yvals[subj_id]
        for comp, comp_res in t['depth']['keypoint_comparisons_res'].iteritems():
            kpt1, kpt2 = [int(_k) for _k in comp.split(',')]
            kpt1_depth = yvals[kpt1]
            kpt2_depth = yvals[kpt2]
            depth_diff = kpt1_depth - kpt2_depth
            if np.sign(depth_diff) == np.sign(comp_res):
                distance_of_correct_comps.append(abs(depth_diff))
            else:
                distance_of_wrong_comps.append(abs(depth_diff))

In [25]:
distance_of_wrong_metacomps = []
distance_of_correct_metacomps = []

for subj_id, t in subj_id_to_metatrials.iteritems():
    yvals = subj_id_to_yvals[subj_id]
    for comp, comp_res in t.iteritems():
        kpt1, kpt2 = [int(_k) for _k in comp.split(',')]
        kpt1_depth = yvals[kpt1]
        kpt2_depth = yvals[kpt2]
        depth_diff = kpt1_depth - kpt2_depth
        if np.sign(depth_diff) == np.sign(comp_res):
            distance_of_correct_metacomps.append(abs(depth_diff))
        else:
            distance_of_wrong_metacomps.append(abs(depth_diff))

In [26]:
subj_id_accuracy = {}
subjects_ranked_hard_to_easy = []

for subj_id in subj_id_to_trials.keys():
    subj_comp_res = []
    trials = subj_id_to_trials[subj_id]
    yvals = subj_id_to_yvals[subj_id]
    for t in trials:
        for comp, comp_res in t.iteritems():
            kpt1, kpt2 = [int(_k) for _k in comp.split(',')]
            kpt1_depth = yvals[kpt1]
            kpt2_depth = yvals[kpt2]
            depth_diff = kpt1_depth - kpt2_depth
            subj_comp_res.append(int(np.sign(depth_diff) == np.sign(comp_res)))
    subj_id_accuracy[subj_id] = sum(subj_comp_res) * 1.0 / len(subj_comp_res)
subjects_ranked_hard_to_easy = [key for key, value in sorted(subj_id_accuracy.iteritems(), key=lambda (k,v): (v,k))]

In [27]:
subj_id_to_image_path = {}

for d in _human_dataset['images']:
    subj_id_to_image_path[d['id']] = d['filename']

In [79]:
# OVERALL ACCURACY
comp_reses = []
for d in data:
    for t in d['trials']:
        subj_id = t['human_subj_id']
        yvals = subj_id_to_yvals[subj_id]
        for comp, comp_res in t['depth']['keypoint_comparisons_res'].iteritems():
            kpt1, kpt2 = [int(_k) for _k in comp.split(',')]
            kpt1_depth = yvals[kpt1]
            kpt2_depth = yvals[kpt2]
            depth_diff = kpt1_depth - kpt2_depth
            comp_reses.append(int(np.sign(depth_diff) == np.sign(comp_res)))
sum(comp_reses) * 1.0 / len(comp_reses)

0.6699489274770174

In [80]:
# OVERALL ACCURACY
depth_diffs = []
for d in data:
    for t in d['trials']:
        subj_id = t['human_subj_id']
        yvals = subj_id_to_yvals[subj_id]
        for comp, comp_res in t['depth']['keypoint_comparisons_res'].iteritems():
            kpt1, kpt2 = [int(_k) for _k in comp.split(',')]
            kpt1_depth = yvals[kpt1]
            kpt2_depth = yvals[kpt2]
            depth_diffs.append(np.sign(kpt1_depth - kpt2_depth))

print("Instances d(kp1) < d(kp2): {}".format(depth_diffs.count(1)))
print("Instances d(kp1) = d(kp2): {}".format(depth_diffs.count(0)))
print("Instances d(kp1) > d(kp2): {}".format(depth_diffs.count(-1)))

Instances d(kp1) < d(kp2): 7708
Instances d(kp1) = d(kp2): 35
Instances d(kp1) > d(kp2): 16732


In [66]:
subj_id_to_kpts_2d = {}

for d in _human_dataset['annotations']:
    subj_id_to_kpts_2d[d['id']] = d['kpts_2d']

In [71]:
subj_id_to_trials

{423936: [{u'0,11': -1, u'11,13': -1, u'2,12': -1, u'3,6': 1, u'5,11': -1},
  {u'0,11': -1, u'11,13': -1, u'2,12': -1, u'3,6': 1, u'5,11': 1},
  {u'0,11': 1, u'11,13': -1, u'2,12': -1, u'3,6': 1, u'5,11': -1},
  {u'0,11': -1, u'11,13': -1, u'2,12': -1, u'3,6': 1, u'5,11': -1},
  {u'0,11': -1, u'11,13': -1, u'2,12': -1, u'3,6': 1, u'5,11': -1}],
 745477: [{u'1,4': 1, u'2,8': 1, u'2,9': 1, u'3,7': -1, u'6,7': -1},
  {u'1,4': 1, u'2,8': -1, u'2,9': 1, u'3,7': -1, u'6,7': -1},
  {u'1,4': 1, u'2,8': -1, u'2,9': -1, u'3,7': -1, u'6,7': -1},
  {u'1,4': 1, u'2,8': -1, u'2,9': 1, u'3,7': -1, u'6,7': -1},
  {u'1,4': 1, u'2,8': -1, u'2,9': 1, u'3,7': -1, u'6,7': -1}],
 827213: [{u'0,11': 1, u'0,5': -1, u'2,8': 1, u'6,8': -1, u'9,10': 1},
  {u'0,11': -1, u'0,5': -1, u'2,8': 1, u'6,8': -1, u'9,10': 1},
  {u'0,11': 1, u'0,5': -1, u'2,8': 1, u'6,8': -1, u'9,10': 1},
  {u'0,11': -1, u'0,5': -1, u'2,8': -1, u'6,8': 1, u'9,10': 1},
  {u'0,11': -1, u'0,5': -1, u'2,8': -1, u'6,8': 1, u'9,10': -1}],
 72500

In [97]:
NUMBER_COMPS_IN_SUBJ = 5
num_pts = 17
num_comps = NUMBER_COMPS_IN_SUBJ
pairs = [(i, j) for i in range(num_pts) for j in range(num_pts) if i < j]
pair_idxs = np.random.choice(len(pairs), num_comps, replace=False)
comps = np.take(pairs, pair_idxs, axis=0).tolist()

for _ci, c in enumerate(comps):
    comps[_ci] = c if bool(random.getrandbits(1)) else [c[1], c[0]]

print comps

[[10, 3], [8, 0], [15, 16], [16, 4], [13, 4]]


In [ ]:
{u'_keypoints_bbox': [0, 0, 256, 256], u'_image_keypoints': [110, 21, 2, 137, 43, 2, 86, 58, 2, 151, 39, 2, 89, 94, 2, 146, 45, 2, 114, 88, 2, 127, 117, 2, 87, 118, 2, 169, 163, 2, 119, 162, 2, 142, 214, 2, 114, 219, 2], u'_comps': [[0, 2], [6, 11], [5, 6], [2, 3], [5, 8]], u'_human_img_src': u'/static/images/human/human36m_train_0000279782.jpg', u'_id': ObjectId('5accb516494f991b4a06bfb0'), u'_human_subj_id': 279782, u'_human_img_id': 279782}